# OBJECTIVES
The goal here is to 
- create a folder 3_aggregats with the same structure as 0_raw.
- each folder must contains one folder with all the data aggregated from **tournaments**, **matches_details**, **matches**.
- handle case where **matches_details** is not present.

# LIBRARIES

In [35]:
from utilities.logger import Logger
from utilities.utilities import *
import datetime
import time
import os
import functools
from typing import *
import pprint
import unidecode
import glob

import pandas as pd

# Set the display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [36]:
#create the folder if not exists
os.makedirs(os.path.join(os.getcwd(), "prd-trd", "3_aggregats"), exist_ok=True)

### Custom functions

In [37]:
from typing import List

def add_empty_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Add new columns with None values to the given DataFrame.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The DataFrame to which new columns will be added.
    columns : List[str]
        The names of the new columns to add.
        
    Returns:
    --------
    pd.DataFrame
        A new DataFrame with new columns added.
    """
    # Creating a DataFrame with None values and the new columns
    empty_columns = pd.DataFrame({col: [None]*len(df) for col in columns})
    
    # Concatenating the original DataFrame and the new columns DataFrame
    new_df = pd.concat([df, empty_columns], axis=1)
    
    return new_df



In [38]:
def convert_to_int_or_none(str_value: str) -> int:
    if str(str_value).isdigit():
        return int(str_value)
    else:
        return None

def calculate_sets_won(row: pd.Series) -> (int, int):
    p1_sets_won, p2_sets_won = 0, 0
    for set_num in range(1, 4):
        try:
            p1_score = convert_to_int_or_none(row[f'player_1_set_{set_num}'])
            p2_score = convert_to_int_or_none(row[f'player_2_set_{set_num}'])
            p1_tiebreak_score = convert_to_int_or_none(row.get(f'player_1_set_{set_num}_tiebreak', None))
            p2_tiebreak_score = convert_to_int_or_none(row.get(f'player_2_set_{set_num}_tiebreak', None))
            
            if p1_tiebreak_score is not None and p2_tiebreak_score is not None:
                p1_sets_won += int(p1_tiebreak_score > p2_tiebreak_score)
                p2_sets_won += int(p2_tiebreak_score > p1_tiebreak_score)
            else:
                if p1_score is not None and p2_score is not None:
                    p1_sets_won += int(p1_score > p2_score)
                    p2_sets_won += int(p2_score > p1_score)
        except Exception as e:
            print(f"Skipping row {row.name} due to {e}.")
            continue
    return p1_sets_won, p2_sets_won

def rename_and_organize_columns(row: pd.Series, winner: str, loser: str) -> dict:
    organized_row = {}
    for col in row.index:
        new_col = col.replace(winner, 'winner').replace(loser, 'loser')
        organized_row[new_col] = row[col]
    return organized_row

def process_tennis_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    new_rows = []
    for i, row in df.iterrows():
        try:
            p1_sets_won, p2_sets_won = calculate_sets_won(row)
            winner, loser = ('player_1', 'player_2') if p1_sets_won > p2_sets_won else ('player_2', 'player_1')
            organized_row = rename_and_organize_columns(row, winner, loser)
            new_rows.append(organized_row)
        except Exception as e:
            print(f"Error while processing row {row.name}: {e}")
            
    return pd.DataFrame.from_records(new_rows)


In [39]:
from typing import Union

def convert_fractions_in_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert fractions in all object columns of a DataFrame to floats.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame containing columns with fractions.
    
    Returns:
    pd.DataFrame: A DataFrame with fractions converted to floats.
    """
    # Identify columns that may contain fractions
    df = df.copy(deep=True)
    potential_fraction_cols = df.select_dtypes(include=['object']).columns
    
    # Nested function to convert fractions to float
    def fraction_to_float(value: Union[str, float, int]) -> Union[float, str]:
        try:
            if '/' in str(value):
                numerator, denominator = map(float, str(value).split('/'))
                return round(numerator / denominator, 2)
            else:
                return round(float(value), 2)
        except Exception:
            return value
    
    # Apply the function to the potential columns containing fractions
    for col in potential_fraction_cols:
        df[col] = df[col].apply(fraction_to_float)
        
    return df


In [40]:
run_id = datetime.datetime.now().strftime('%Y%m%d_%H%M')
print(f"run id: {run_id}")
logger = Logger(run_id)
logger.initialization()

run id: 20230925_1207


## Optimized version of my class

In [65]:
# Applying modifications, adding type annotations, and docstrings to the class
import os
import pandas as pd
from typing import List, Dict, Union, Optional

class AggregateExistingTournamentData:
    """
    Create 3_aggregats folder and reproduce 0_raw folder hierarchy with 
    matches and matches_details aggregated.
    """
    
    base_path_aggregat: str = os.path.join(os.getcwd(), "prd-trd", "3_aggregats")

    def __init__(self, 
                 years: Union[List[str], None], 
                 logger: Logger, 
                 whole_stack: bool = False) -> None:
        """
        Initialize AggregateExistingTournamentData instance.
        
        Parameters:
        -----------
        years : Union[List[str], None], optional
            The years to consider for data aggregation.
        logger : Logger
            The logger instance for logging information.
        whole_stack : bool, optional
            Whether to process the whole data stack.
        """
        self.years_selected = years if years is not None else os.listdir(os.path.join(os.getcwd(), "prd-trd", "0_raw"))
        self.whole_stack = whole_stack
        self.tourney_path_folder: List[str] = []
        self.tourney_data: Dict[str, Dict] = {}
        self.logger = logger
        self.aggregated_data: Dict[str, pd.DataFrame] = {}
        self.aggregated_data_ratio: Dict[str, pd.DataFrame] = {}
        self.logger.log_info(f"years selected: {', '.join(self.years_selected)}")

    def list_tourney_folder(self) -> None:
        """
        List tournament folders based on the years selected.
        """
        base_path = os.path.join(os.getcwd(), "prd-trd", "0_raw")
        for year in self.years_selected:
            year_path = os.path.join(base_path, year)
            tourneys_for_year = os.listdir(year_path)
            for tourney in tourneys_for_year:
                self.tourney_path_folder.append(os.path.join(year_path, tourney))

        self.logger.log_info(f"len tourney: {len(self.tourney_path_folder)}")
    
    def compute_data_in_folders(self) -> None:
        """
        Compute data in each tournament folder.
        """
        for path in self.tourney_path_folder:
            path_tournament = path.replace("0_raw", "1_data_quality")
            tournament_name = os.path.basename(path)
            tournament = pd.read_csv(os.path.join(path_tournament, f"{tournament_name}.csv"), sep=";")
            matches = self._read_csv_with_fallback(path, "matches.csv")
            matches_details = self._read_csv_with_fallback(path, "matches_details_extended.csv", "matches_details.csv")
            self.tourney_data[path] = {"tournament": tournament, "matches": matches, "matches_details": matches_details}

    def _read_csv_with_fallback(self, path: str, filename: str, fallback_filename: Optional[str] = None) -> Optional[pd.DataFrame]:
        """
        Read a CSV file and return a DataFrame. Fallback to another file if the main file is not found.
        
        Parameters:
        -----------
        path : str
            The directory where the file is located.
        filename : str
            The name of the file to read.
        fallback_filename : Optional[str], optional
            The name of the fallback file to read if the main file is not found.
        
        Returns:
        --------
        Optional[pd.DataFrame]
            The DataFrame read from the file, or None if neither file is found.
        """
        try:
            return pd.read_csv(os.path.join(path, filename), sep=";")
        except Exception as e:
            self.logger.log_info(f"No {filename} in path {e}")
            if fallback_filename:
                try:
                    return pd.read_csv(os.path.join(path, fallback_filename), sep=";")
                except Exception as e:
                    self.logger.log_info(f"No {fallback_filename} in path {e}")
                    return None
            return None

    def computing_aggregating_data(self) -> None:
        """
        Compute and aggregate data for each tournament.
        """
        for path, tourney_data in self.tourney_data.items():
            tournament_light = tourney_data["tournament"][["title", "location", "scores_link", "surface", "date_debut", "date_fin"]]
            #handling matches and matches_details null cases
            matches_details_cols = ["match_1st Return Points Won_player_1","match_1st Return Points Won_player_2",
                                    "match_1st Serve Points Won_player_1","match_1st Serve Points Won_player_2",
                                    "match_1st Serve_player_1","match_1st Serve_player_2",
                                    "match_2nd Return Points Won_player_1","match_2nd Return Points Won_player_2",
                                    "match_2nd Serve Points Won_player_1","match_2nd Serve Points Won_player_2",
                                    "match_Aces_player_1","match_Aces_player_2","match_Break Points Converted_player_1",
                                    "match_Break Points Converted_player_2","match_Break Points Saved_player_1",
                                    "match_Break Points Saved_player_2","match_Double Faults_player_1",
                                    "match_Double Faults_player_2","match_Return Games Played_player_1",
                                    "match_Return Games Played_player_2","match_Service Games Played_player_1",
                                    "match_Service Games Played_player_2","match_Total Points Won_player_1",
                                    "match_Total Points Won_player_2","match_Total Return Points Won_player_1",
                                    "match_Total Return Points Won_player_2","match_Total Service Points Won_player_1",
                                    "match_Total Service Points Won_player_2","set1_1st Return Points Won_player_1",
                                    "set1_1st Return Points Won_player_2","set1_1st Serve Points Won_player_1",
                                    "set1_1st Serve Points Won_player_2","set1_1st Serve_player_1","set1_1st Serve_player_2",
                                    "set1_2nd Return Points Won_player_1","set1_2nd Return Points Won_player_2",
                                    "set1_2nd Serve Points Won_player_1","set1_2nd Serve Points Won_player_2",
                                    "set1_Aces_player_1","set1_Aces_player_2","set1_Break Points Converted_player_1",
                                    "set1_Break Points Converted_player_2","set1_Break Points Saved_player_1",
                                    "set1_Break Points Saved_player_2","set1_Double Faults_player_1",
                                    "set1_Double Faults_player_2","set1_Return Games Played_player_1",
                                    "set1_Return Games Played_player_2","set1_Service Games Played_player_1",
                                    "set1_Service Games Played_player_2","set1_Total Points Won_player_1",
                                    "set1_Total Points Won_player_2","set1_Total Return Points Won_player_1",
                                    "set1_Total Return Points Won_player_2","set1_Total Service Points Won_player_1",
                                    "set1_Total Service Points Won_player_2","set2_1st Return Points Won_player_1",
                                    "set2_1st Return Points Won_player_2","set2_1st Serve Points Won_player_1",
                                    "set2_1st Serve Points Won_player_2","set2_1st Serve_player_1",
                                    "set2_1st Serve_player_2","set2_2nd Return Points Won_player_1",
                                    "set2_2nd Return Points Won_player_2","set2_2nd Serve Points Won_player_1",
                                    "set2_2nd Serve Points Won_player_2","set2_Aces_player_1","set2_Aces_player_2",
                                    "set2_Break Points Converted_player_1","set2_Break Points Converted_player_2",
                                    "set2_Break Points Saved_player_1","set2_Break Points Saved_player_2",
                                    "set2_Double Faults_player_1","set2_Double Faults_player_2",
                                    "set2_Return Games Played_player_1","set2_Return Games Played_player_2",
                                    "set2_Service Games Played_player_1","set2_Service Games Played_player_2",
                                    "set2_Total Points Won_player_1","set2_Total Points Won_player_2",
                                    "set2_Total Return Points Won_player_1","set2_Total Return Points Won_player_2",
                                    "set2_Total Service Points Won_player_1","set2_Total Service Points Won_player_2",
                                    "h2h_wins_player_1","h2h_wins_player_2","Height_player_1","Height_player_2",
                                    "Date of Birth_player_1","Date of Birth_player_2","Plays_player_1","Plays_player_2",
                                    "url_match_stats","set3_1st Return Points Won_player_1","set3_1st Return Points Won_player_2",
                                    "set3_1st Serve Points Won_player_1","set3_1st Serve Points Won_player_2","set3_1st Serve_player_1",
                                    "set3_1st Serve_player_2","set3_2nd Return Points Won_player_1","set3_2nd Return Points Won_player_2",
                                    "set3_2nd Serve Points Won_player_1","set3_2nd Serve Points Won_player_2","set3_Aces_player_1",
                                    "set3_Aces_player_2","set3_Break Points Converted_player_1","set3_Break Points Converted_player_2",
                                    "set3_Break Points Saved_player_1","set3_Break Points Saved_player_2","set3_Double Faults_player_1",
                                    "set3_Double Faults_player_2","set3_Return Games Played_player_1","set3_Return Games Played_player_2",
                                    "set3_Service Games Played_player_1","set3_Service Games Played_player_2","set3_Total Points Won_player_1",
                                    "set3_Total Points Won_player_2","set3_Total Return Points Won_player_1","set3_Total Return Points Won_player_2",
                                    "set3_Total Service Points Won_player_1","set3_Total Service Points Won_player_2"]

            if all(tourney_data.get(key) is not None for key in ["matches", "matches_details"]):
                self.logger.log_info(f"tourney: {path}")
                self.logger.log_info("case matches and matches_details not null")
                matches = tourney_data["matches"]
                self.logger.log_info(f"matches columns: {', '.join(list(matches.columns))}")
                matches_details = tourney_data["matches_details"]
                self.logger.log_info(f"matches details columns: {', '.join(list(matches_details.columns))}")
                matches_total = pd.merge(matches, tournament_light, on="scores_link", how="left")
                self.logger.log_info(f"matches total after merge with tournament light: {', '.join(list(matches_total.columns))}")
                matches_total = pd.merge(matches_total, matches_details, on="url_match_stats", how="left")
                self.logger.log_info(f"matches total after merge with matches details: {', '.join(list(matches_total.columns))}")
                # Assuming process_tennis_row and convert_fractions_in_dataframe are defined elsewhere
                matches_total = process_tennis_dataframe(matches_total)
                self.logger.log_info(f"after process_tennis_row: {', '.join(list(matches_total.columns))}")
                matches_total_ratio = convert_fractions_in_dataframe(matches_total)
                self.logger.log_info(f"after ratio: {', '.join(list(matches_total.columns))}")
                self.aggregated_data[path] = matches_total
                self.aggregated_data_ratio[path] = matches_total_ratio
            elif tourney_data["matches"] is not None and tourney_data["matches_details"] is None:
                self.logger.log_info(f"tourney: {path}")
                self.logger.log_info("case matches not None and matches_details None")
                matches = tourney_data["matches"]
                matches_details = tourney_data["matches_details"]
                matches_total = pd.merge(matches, tournament_light, on="scores_link", how="left")
                self.logger.log_info(f"matches total after merge with tourn light: {', '.join(list(matches_total.columns))}")
                # Assuming process_tennis_row and convert_fractions_in_dataframe are defined elsewhere
                matches_total = process_tennis_dataframe(matches_total)
                self.logger.log_info(f"matches total after processing tennis: {', '.join(list(matches_total.columns))}")
                matches_total = add_empty_columns(matches_total, matches_details_cols)
                self.aggregated_data[path] = matches_total
                self.logger.log_info(f"matches total after empty data: {', '.join(list(matches_total.columns))}")
                matches_total_ratio = None 
                self.aggregated_data_ratio[path] = matches_total_ratio
                
            self.logger.log_info("="*200)


    
    @classmethod
    def set_aggregat_path(cls, path: str) -> None:
        """
        Set the base aggregation path for the class.
        
        Parameters:
        -----------
        path : str
            The new base aggregation path.
        """
        cls.base_path_aggregat = path

    def saving_data(self) -> None:
        """
        Save the aggregated data to CSV files.
        """
        for path, data in self.aggregated_data.items():
            path_corrected = path.replace("0_raw", "3_aggregats")
            self.logger.log_info(f"Creating folder: {path_corrected}")
            os.makedirs(path_corrected, exist_ok=True)
            writting_path = os.path.join(f"{path_corrected}", "aggregated_matches_details.csv")
            self.logger.log_info(f"Writting to: {writting_path}")
            if data is not None:
                data.to_csv(writting_path, sep=";", index=False)
            elif self.aggregated_data_ratio[path] is not None:
                writting_path_ratio = os.path.join(f"{path_corrected}", "aggregated_matches_details_ratio.csv")
                self.aggregated_data_ratio[path].to_csv(writting_path_ratio, sep=";", index=False)




In [66]:
aggregated = AggregateExistingTournamentData(["2023"], logger)

years selected: 2023


In [67]:
aggregated.list_tourney_folder()

len tourney: 62


In [68]:
aggregated.compute_data_in_folders()

No matches_details_extended.csv in path [Errno 2] No such file or directory: 'c:\\Users\\beAtz\\OneDrive\\Bureau\\WTA_PROJECT\\prd-trd\\0_raw\\2023\\34_Palermo_Ladies_Open\\matches_details_extended.csv'
No matches_details_extended.csv in path [Errno 2] No such file or directory: 'c:\\Users\\beAtz\\OneDrive\\Bureau\\WTA_PROJECT\\prd-trd\\0_raw\\2023\\Abierto_GNP_Seguros\\matches_details_extended.csv'
No matches_details_extended.csv in path [Errno 2] No such file or directory: 'c:\\Users\\beAtz\\OneDrive\\Bureau\\WTA_PROJECT\\prd-trd\\0_raw\\2023\\Australian_Open\\matches_details_extended.csv'
No matches_details.csv in path [Errno 2] No such file or directory: 'c:\\Users\\beAtz\\OneDrive\\Bureau\\WTA_PROJECT\\prd-trd\\0_raw\\2023\\Australian_Open\\matches_details.csv'
No matches_details_extended.csv in path [Errno 2] No such file or directory: 'c:\\Users\\beAtz\\OneDrive\\Bureau\\WTA_PROJECT\\prd-trd\\0_raw\\2023\\Bad_Homburg_Open_presented_by_Engel_&_Volkers\\matches_details_extended.cs

In [70]:
aggregated.computing_aggregating_data()

tourney: c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\0_raw\2023\34_Palermo_Ladies_Open
case matches and matches_details not null
matches columns: statut, duration, stage, url_match_stats, player_1_name, player_1_set_1, player_1_set_1_tiebreak, player_1_set_2, player_1_set_2_tiebreak, player_1_set_3, player_1_set_3_tiebreak, player_2_name, player_2_set_1, player_2_set_1_tiebreak, player_2_set_2, player_2_set_2_tiebreak, player_2_set_3, player_2_set_3_tiebreak, day, scores_link
matches details columns: match_1st Return Points Won_player_1, match_1st Return Points Won_player_2, match_1st Serve Points Won_player_1, match_1st Serve Points Won_player_2, match_1st Serve_player_1, match_1st Serve_player_2, match_2nd Return Points Won_player_1, match_2nd Return Points Won_player_2, match_2nd Serve Points Won_player_1, match_2nd Serve Points Won_player_2, match_Aces_player_1, match_Aces_player_2, match_Break Points Converted_player_1, match_Break Points Converted_player_2, match_Break Poi

In [72]:
aggregated.aggregated_data['c:\\Users\\beAtz\\OneDrive\\Bureau\\WTA_PROJECT\\prd-trd\\0_raw\\2023\\US_Open']

statut  duration           stage  \
0    Completed NaN        Qualifying       
1    Completed NaN        Qualifying       
2    Completed NaN        Qualifying       
3    Completed NaN        Qualifying       
4    Completed NaN        Qualifying       
5    Completed NaN        Qualifying       
6    Completed NaN        Qualifying       
7    Completed NaN        Qualifying       
8    Completed NaN        Qualifying       
9    Completed NaN        Qualifying       
10   Completed NaN        Qualifying       
11   Completed NaN        Qualifying       
12   Completed NaN        Qualifying       
13   Completed NaN        Qualifying       
14   Completed NaN        Qualifying       
15   Completed NaN        Qualifying       
16   Completed NaN        Qualifying       
17   Completed NaN        Qualifying       
18   Completed NaN        Qualifying       
19   Completed NaN        Qualifying       
20   Completed NaN        Qualifying       
21   Completed NaN        Qualifying       
22   Completed NaN        Qualifying       
23   Completed NaN        Final            
24   Completed NaN        Qualifying       
25   Completed NaN        Qualifying       
26   Completed NaN        Qualifying       
27   Completed NaN        Qualifying       
28   Completed NaN        Qualifying       
29   Completed NaN        Qualifying       
30   Completed NaN        Qualifying       
31   Completed NaN        Qualifying       
32   Completed NaN        Qualifying       
33   Completed NaN        Qualifying       
34   Completed NaN        Qualifying       
35   Completed NaN        Qualifying       
36   Completed NaN        Qualifying       
37   Completed NaN        Qualifying       
38   Completed NaN        Qualifying       
39   Completed NaN        Final            
40   Completed NaN        Qualifying       
41   Completed NaN        Qualifying       
42   Completed NaN        Qualifying       
43   Completed NaN        Qualifying       
44   Completed NaN        Qualifying       
45   Completed NaN        Qualifying       
46   Completed NaN        Qualifying       
47   Completed NaN        Qualifying       
48   Completed NaN        Qualifying       
49   Completed NaN        Final            
50   Completed NaN        Round of 128     
51   Completed NaN        Round of 128     
52   Completed NaN        Round of 128     
53   Completed NaN        Round of 128     
54   Completed NaN        Round of 128     
55   Completed NaN        Round of 128     
56   Completed NaN        Round of 128     
57   Completed NaN        Round of 128     
58   Completed NaN        Round of 128     
59   Completed NaN        Round of 128     
60   Completed NaN        Round of 128     
61   Completed NaN        Round of 128     
62   Completed NaN        Round of 128     
63   Completed NaN        Round of 128     
64   Completed NaN        Round of 128     
65   Completed NaN        Round of 128     
66   Cancelled NaN        Round of 128     
67   Completed NaN        Round of 128     
68   Completed NaN        Round of 128     
69   Completed NaN        Round of 128     
70   Completed NaN        Round of 128     
71   Completed NaN        Round of 128     
72   Completed NaN        Round of 128     
73   Completed NaN        Round of 128     
74   Completed NaN        Round of 128     
75   Completed NaN        Round of 128     
76   Completed NaN        Round of 128     
77   Completed NaN        Round of 128     
78   Completed NaN        Round of 128     
79   Completed NaN        Round of 128     
80   Completed NaN        Round of 128     
81   Completed NaN        Round of 128     
82   Completed NaN        Final            
83   Completed NaN        Round of 128     
84   Completed NaN        Round of 128     
85   Completed NaN        Round of 128     
86   Completed NaN        Round of 128     
87   Cancelled NaN        Round of 128     
88   Completed NaN        Round of 128     
89   Completed NaN        Round of 128     
90  

# END

In [ ]:
class AggregateExistingTournamentData:
    """
    create 3_aggregats folder and reproduce
    0_raw folder hierarchy with matches and 
    matches_details aggregated
    """
    base_path_aggregat = os.path.join(os.getcwd(), "prd-trd", "3_aggregats")

    def __init__(self, years, logger, whole_stack=False):
        if isinstance(years, list):
            self.years_selected = years
        else:
            self.years_selected = os.listdir(os.path.join(os.getcwd(),"prd-trd", "0_raw"))

        self.whole_stack = whole_stack
        self.tourney_path_folder = []
        self.tourney_data = dict()
        self.logger = logger
        self.aggregated_data = dict()
        self.aggregated_data_ratio = dict()
        self.logger.log_info(f"years selected: {', '.join(years)}")
    def list_tourney_folder(self):
        """
        given
        """
        base_path = os.path.join(os.getcwd(),"prd-trd", "0_raw")

        for year in self.years_selected:
            year_path = os.path.join(base_path, year)
            tourneys_for_year = os.listdir(year_path)
            for tourney in tourneys_for_year:
                self.tourney_path_folder.append(os.path.join(year_path, tourney))
    

    
    def compute_data_in_folders(self):
        """
        define what we want to define
        """
        # initialize the 
        for path in self.tourney_path_folder:
            path_tournament = path.replace("0_raw", "1_data_quality")
            tournament_name = path.split("\\")[-1] 
            tournament = pd.read_csv(os.path.join(path_tournament, f"{tournament_name}.csv"), sep=";")
            #selection match_details ot match_details_extended
            try:
                matches = pd.read_csv(os.path.join(path, "matches.csv"), sep=";")
            except Exception as e:
                print(f"no matches in path {e}")
                matches = None

            try:
                matches_details = pd.read_csv(os.path.join(path, "matches_details.csv"), sep=";")
            except Exception as e:
                print(f"no matches details in path {e}")
                matches_details = None

            try:
                matches_details = pd.read_csv(os.path.join(path, "matches_details_extended.csv"), sep=";")
            except Exception as e:
                matches_details = None
                
            self.tourney_data[path] = {"tournament": tournament,
                                       "matches": matches,
                                       "matches_details": matches_details
                                       }
    
    def computing_aggregating_data(self):
        """
        0_raw.ipynb
        """
        for path, tourney_data in self.tourney_data.items():
            tournament_light = tourney_data["tournament"][["title", "location", "scores_link",
                                                            "surface", "date_debut", "date_fin"]]
            matches = tourney_data["matches"]

            matches_details = tourney_data["matches_details"]
            if matches is not None and matches_detail
            #merge tournament and matches
            matches_total = pd.merge(matches, tournament_light, on="scores_link", how="left")

            #merge [tournament, matches] with matches_details
            matches_total = pd.merge(matches_total, matches_details, on= "url_match_stats", how="left")

            #apply transformation on data
            matches_total.apply(process_tennis_row, axis=1)
            #modify fraction to ratio
            matches_total_ratio = convert_fractions_in_dataframe(matches_total)
            #added
            self.aggregated_data[path] = matches_total
            #no ratio
            self.aggregated_data_ratio[path] = matches_total_ratio
    
    @classmethod
    def set_aggregat_path(cls, path):
        cls.base_path_aggregat = path

    def saving_data(self):
        for path, data in self.aggregated_data.items():
            path_corrected = path.replace("0_raw", "3_aggregats")
            self.logger.log_info(f"creating folder: {path_corrected}")
            os.makedirs(path_corrected, exist_ok=True)
            writting_path = os.path.join(f"{path_corrected}", "aggregated_matches_details.csv")
            self.logger.log_info(f"writting to: {writting_path}")
            data.to_csv(writting_path, sep=";", index=False)



